# Importing Libraries

In [10]:
import numpy as np
import matplotlib.pyplot as plt
import os
import time
import pydot
import tempfile
import graphviz
import pandas as pd
from tf_keras import optimizers

import tensorflow as tf
import tensorflow_datasets as tfds
import keras

# Importing the dataset Horses or Humans

In [11]:
(train_examples, validation_examples), info = tfds.load(
    'horses_or_humans',
    split = ('train[:70%]', 'train[70%:]'),
    with_info = True,
    as_supervised = True
)

In [12]:
# Information about the dataset Horses or Humans
num_examples = info.splits['train'].num_examples
num_classes = info.features['label'].num_classes
class_names = info.features['label'].names
num_examples,num_classes, class_names

(1027, 2, ['horses', 'humans'])

# Converting the images to the required size for TensorFlow Model

In [13]:
def format_image(image, label):
  image = tf.image.resize(image, IMG_SIZE)
  return image,label

BATCH_SIZE = 32
IMG_SIZE = (224, 224)

train_batches = train_examples.cache().shuffle(num_examples//4).map(format_image).batch(BATCH_SIZE).prefetch(1)
validation_batches = validation_examples.map(format_image).batch(BATCH_SIZE).prefetch(1)

In [14]:
train_batches

<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int64, name=None))>

# Creating the Model using Transfer Learning with VGG16 Model and Personal Classification Layers

In [15]:
from tf_keras.layers import Dense, Flatten, Dropout, GlobalAveragePooling2D
from tf_keras.models import Model
from tf_keras.applications import VGG16
from tf_keras import layers
from tf_keras import Sequential
from tf_keras.layers import Input
from tf_keras import losses

import tensorflow_model_optimization as tfmot

# Create the base model from the pre-trained model VGG16 for transfer learning
vgg16_model = VGG16(input_shape=(224, 224, 3),
                    include_top=False,
                    weights='imagenet')
# Freeze the base model
vgg16_model.trainable = False

# Create a sequential mode
test_model = Sequential()
for layer in vgg16_model.layers[:-4]:
    test_model.add(layer)

del vgg16_model

for layer in test_model.layers:
    layer.trainable = False


# Add personal classification layers on top of the base model
max_pool_layer = layers.GlobalMaxPooling2D()
prediction_layer = layers.Dense(2, activation='softmax')  # For binary classification (2 classes)

inputs = Input(shape=(224, 224, 3))
x = test_model(inputs)  # Use default training=True behavior
x = max_pool_layer(x)
x = layers.Dropout(0.3)(x)  # Apply dropout during training
outputs = prediction_layer(x)
model = Model(inputs, outputs, name='model_horses_or_humans.keras')

model.summary()

Model: "model_horses_or_humans.keras"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 sequential_1 (Sequential)   (None, 14, 14, 512)       7635264   
                                                                 
 global_max_pooling2d_1 (Gl  (None, 512)               0         
 obalMaxPooling2D)                                               
                                                                 
 dropout_1 (Dropout)         (None, 512)               0         
                                                                 
 dense_1 (Dense)             (None, 2)                 1026      
                                                                 
Total params: 7636290 (29.13 MB)
Trainable params: 1026 (4.01 KB)
Non-trainable params: 7635264 (29.13 M

# Compile the model

In [16]:
from compilation import create_model_checkpoint

In [17]:
model.compile(
    optimizer = 'adam',
    loss = losses.SparseCategoricalCrossentropy(),
    metrics=['accuracy'])


In [18]:
EPOCHS = 3
history = model.fit(train_batches,
                    epochs = EPOCHS,
                    batch_size = BATCH_SIZE,
                    validation_data = validation_batches)

Epoch 1/3



23/23 [==============================] - 112s 5s/step - loss: 507.0986 - accuracy: 0.4951 - val_loss: 292.8456 - val_accuracy: 0.5260
Epoch 2/3
23/23 [==============================] - 105s 5s/step - loss: 362.3079 - accuracy: 0.5508 - val_loss: 41.9744 - val_accuracy: 0.8214
Epoch 3/3
23/23 [==============================] - 106s 5s/step - loss: 234.8583 - accuracy: 0.6565 - val_loss: 17.9485 - val_accuracy: 0.9156


In [19]:
test_model.trainable = True
# Fine-tune from this layer onwards
fine_tune_at = 2

# Freeze all the layers before the `fine_tune_at` layer
for layer in test_model.layers[:-fine_tune_at]:
  layer.trainable =  False

In [20]:
model.compile(loss=losses.SparseCategoricalCrossentropy(),
              optimizer = optimizers.Adam(learning_rate=0.0001),
              metrics=['accuracy'])

In [21]:
EPOCHS = 5
history = model.fit(train_batches,
                    epochs = EPOCHS,
                    batch_size=BATCH_SIZE,
                    validation_data = validation_batches,
                    callbacks=[create_model_checkpoint(model_name=model.name)])

Epoch 1/5
23/23 [==============================] - 119s 5s/step - loss: 102.6089 - accuracy: 0.8414 - val_loss: 3.1261 - val_accuracy: 0.9838
Epoch 2/5
23/23 [==============================] - 116s 5s/step - loss: 26.0901 - accuracy: 0.9513 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 3/5
23/23 [==============================] - 126s 6s/step - loss: 6.8400 - accuracy: 0.9847 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 4/5
23/23 [==============================] - 118s 5s/step - loss: 4.9745 - accuracy: 0.9917 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 5/5
23/23 [==============================] - 117s 5s/step - loss: 2.4566 - accuracy: 0.9917 - val_loss: 0.0000e+00 - val_accuracy: 1.0000


In [22]:
from tf_keras.utils import plot_model
import pydot
import graphviz
plot_model(model, to_file="horses_or_humans_vgg16.jpeg",show_shapes=True, show_dtype=True)

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model to work.


## Save the model and history

In [23]:
from tf_keras import models
model_1 = models.load_model('model_experiments/model_horses_or_humans.keras')
history_1 = history.history

In [24]:
model_1.evaluate(validation_batches)

10/10 [==============================] - 32s 3s/step - loss: 0.0000e+00 - accuracy: 1.0000


[0.0, 1.0]

In [25]:
model.save_weights("save_weights.weights.h5")

In [26]:
model_1.summary()

Model: "model_horses_or_humans.keras"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 sequential_1 (Sequential)   (None, 14, 14, 512)       7635264   
                                                                 
 global_max_pooling2d_1 (Gl  (None, 512)               0         
 obalMaxPooling2D)                                               
                                                                 
 dropout_1 (Dropout)         (None, 512)               0         
                                                                 
 dense_1 (Dense)             (None, 2)                 1026      
                                                                 
Total params: 7636290 (29.13 MB)
Trainable params: 2360834 (9.01 MB)
Non-trainable params: 5275456 (20.1

In [27]:
val_data = validation_examples
val_data = val_data.map(format_image).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
print(val_data)

<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int64, name=None))>


In [28]:
def normalize_img(image, label):
  """Normalizes images: `uint8` -> `float32`."""
  return tf.cast(image, tf.float32), label

val_data = val_data.map(normalize_img)
val_data

<_MapDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int64, name=None))>

# Evaluate the model on the test data

In [29]:
baseline_score = model_1.evaluate(validation_batches)
print(f"The accuracy of the baseline model is {baseline_score[1] * 100:.2f} %")
base_metrics = {
      "sparsity" : 0,
      "val_loss" : np.round(baseline_score[0], 4),
      "val_accuracy" : np.round(baseline_score[1] * 100, 4)
}
base_metrics

10/10 [==============================] - 32s 3s/step - loss: 0.0000e+00 - accuracy: 1.0000
The accuracy of the baseline model is 100.00 %


{'sparsity': 0, 'val_loss': 0.0, 'val_accuracy': 100.0}

## Save the model and history


In [30]:
import PIL.Image as Image

print(val_data)

def benchmark(model, class_names=class_names, image_size=IMG_SIZE):
  file_count = 0
  infer_times = []
  init_timer = 0

  for image in val_data:
    if file_count < 1 :
        init_timer_start = time.time()
        pred = model.predict(image[0])
        pred_class = class_names[int(np.argmax(pred[0]))]
        init_timer_end = time.time()
        init_timer = init_timer_end - init_timer_start
        file_count+=1
    else:
        timer_start = time.time()
        pred = model.predict(image[0])
        pred_class = class_names[int(np.argmax(pred[0]))]
        timer_end = time.time()
        infer_times.append((timer_end - timer_start))
        file_count+=1

  return init_timer, np.mean(infer_times), np.std(infer_times)

<_MapDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int64, name=None))>


In [31]:
init_time, avg_time, std = benchmark(model=model_1)
print(f"The first image takes {init_time * 1000:.2f} ms")
print(f"The average time taken per 99 images {avg_time * 1000:.2f} ms")
print(f"The standard deviation of samples is {std * 1000:.2f} ms")

1/1 [==============================] - 4s 4s/step


KeyboardInterrupt: 

In [32]:
import tensorflow_model_optimization.python.core.sparsity as sparsity
import tensorflow_model_optimization as tfmot


# Finish pruning after 2 epochs
epochs = 2
BATCH_SIZE = 32

In [33]:

def prune_model(model, initial_sparsity, final_sparsity, train_data=train_batches, val_data=validation_batches, epochs=epochs, feature_extractor=model ):

  # Create a tensorboard logfile
  logdir = tempfile.mkdtemp()
  # The end_step is the total number of iterations required for the training data which is basically the entire epochs over the length of the training data
  end_step = int(len(train_data) * epochs * 0.5)
  # Import the low-magnitude-pruning function
  prune_low_magnitude = tfmot.sparsity.keras.prune_low_magnitude

  # Set the prunung params
  pruning_params = {

      "pruning_schedule" : tfmot.sparsity.keras.PolynomialDecay(initial_sparsity=0,
                                                                final_sparsity=final_sparsity,
                                                                begin_step=0,
                                                                end_step=end_step)

                  }


  learning_rate_fn = keras.optimizers.schedules.PolynomialDecay(
    0.001,
    1000,
    0.0001,
    power=0.5)

  # Model for pruning
  #feature_extractor = prune_low_magnitude(feature_extractor, **pruning_params)
  model_for_pruning = prune_low_magnitude(model, **pruning_params)

  # Recompile
  model_for_pruning.compile(optimizer= optimizers.Adam(),
                            loss= losses.SparseCategoricalCrossentropy(),
                            metrics=["accuracy"])
  #create callbacks
  callbacks = [tfmot.sparsity.keras.UpdatePruningStep(),
              tfmot.sparsity.keras.PruningSummaries(log_dir=logdir),
               #create_model_checkpoint(model_name=model.name),
              #tf.keras.callbacks.ReduceLROnPlateau(monitor="val_loss",
                                                  #patience=3,
                                                  #verbose=1),
               #tf.keras.callbacks.EarlyStopping(monitor="val_loss",
                                                        #patience=4,
                                                        #restore_best_weights=True)
                                                        ]

  # Fit the model
  model_for_pruning.fit(train_data,
                      validation_data=val_data,
                      batch_size=BATCH_SIZE,
                      epochs=epochs,
                      callbacks=callbacks)

  # Save the model
  #model_for_pruning.save(f"mnist_model_sparsity_{final_sparsity}")

  # Evaluate the model
  score = model_for_pruning.evaluate(val_data, verbose=0)
  metric_dict = {
      "sparsity" : final_sparsity,
      "val_loss" : np.round(score[0], 4),
      "val_accuracy" : np.round(score[1] * 100, 4)
  }
  return logdir, metric_dict, model_for_pruning

In [34]:
from tf_keras import models
k_sparsities = [0.25, 0.50, 0.60, 0.70, 0.80, 0.90, 0.95, 0.97, 0.99]

#k_sparsities = [0.50]
metric_list = []

for k in k_sparsities:
  # Load in the best saved model
  model_1 = models.load_model("model_experiments/model_horses_or_humans.keras")
  logdir, metrics, pruned_model = prune_model(model=model_1,
            initial_sparsity=0,
            final_sparsity=k,
            epochs=20)
  val_loss, val_accuracy  = metrics["val_loss"], metrics["val_accuracy"]
  metric_list.append(metrics)
  print(f"Sparsity : {k} \tValidation Loss: {val_loss}, \tValidation Accuracy: {val_accuracy}")

# Create a dataframe of the values obtained
df = pd.DataFrame(metric_list)

Epoch 1/20
23/23 [==============================] - 122s 5s/step - loss: 98.8900 - accuracy: 0.9221 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 2/20
23/23 [==============================] - 118s 5s/step - loss: 1.1279 - accuracy: 0.9958 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 3/20
23/23 [==============================] - 115s 5s/step - loss: 0.1049 - accuracy: 0.9986 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 4/20
23/23 [==============================] - 116s 5s/step - loss: 7.9411 - accuracy: 0.9944 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 5/20
23/23 [==============================] - 115s 5s/step - loss: 2.5023 - accuracy: 0.9986 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 6/20
23/23 [==============================] - 116s 5s/step - loss: 1.2839 - accuracy: 0.9986 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 7/20
23/23 [==============================] - 117s 5s/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.7402 - val_a

In [35]:
df

,sparsity,val_loss,val_accuracy
0,0.25,0.0000,100.0000
1,0.50,0.0000,100.0000
2,0.60,0.0000,100.0000
3,0.70,0.0000,100.0000
4,0.80,0.0000,100.0000
5,0.90,0.7021,48.7013
6,0.95,0.6898,49.6753
7,0.97,0.6988,49.6753
8,0.99,0.6889,50.3247


In [36]:
# Save the dataframe
df.to_csv("pruning_metrics.csv")